<a href="https://colab.research.google.com/github/paulaassisb/projeto-final-ds-awari/blob/main/Projeto_Awari_Paula_Assis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2415]:
## Importação de bibliotecas
import pandas as pd
from pandas import Series,DataFrame
import numpy as np
import re
import unicodedata
from nltk.corpus import stopwords
## import matplotlib.pyplot as plt

In [2416]:
## Importando arquivo da coleta de dados
url = 'https://raw.githubusercontent.com/paulaassisb/projeto-final-ds-awari/main/coleta_dados_quest_projeto_final.csv'
dados_quest = pd.read_csv(url)
print(f'Número de linhas: {len(dados_quest)}')
print(f'Número de colunas: {len(dados_quest.columns)}')


Número de linhas: 2780
Número de colunas: 21


In [2417]:
## Transformando em DataFrame para visualização e manipulação dos dados
df = pd.DataFrame(dados_quest)

In [2418]:
df.head()

,Numero do Sujeito,1a. Doença,Idade 1a. Doença,Tempo 1a Doença,Sexo,UF Nascimento,UF,Formação,Atividade,Escolaridade,Estado Civil,Religião,Outro Idioma,Análise do Contexto,Autoconfiança,Autocontrole,Conquistar e Manter Pessoas,Empatia,Leitura Corporal,Otimismo com a Vida,Sentido da Vida
0,13531,depressão,31 - 49 anos,Um ano,Feminino,SP,SP,Administradora,Gerente de Compras,Pós-Graduação,Casado,Católico,Sim,Boa no estresse (PC-P),Fraca no estresse (PC-I),Fraca no estresse (PC-I),Boa diante do impacto do estresse (PC-P),Moderada no estresse (PC-I),Excelente no estresse,Forte no estresse (PC-P),Boa no estresse (PC-P)
1,13532,enfermidade que me levou a 15 dias de hospital...,7 - 12 anos,Mais de um ano,Masculino,SP,SP,Arquiteto,NaN,Pós-Graduação,Casado,Evangélico,Sim,Excelente no estresse,Moderada no estresse (PC-I),Fraca no estresse (PC-I),Boa diante do impacto do estresse (PC-P),Boa no estresse (PC-P),Boa no estresse (PC-P),Excelente no estresse,Moderada no estresse (PC-P)
2,13533,Violência dentro de casa,7 - 12 anos,Mais de um ano,Masculino,SP,SP,NaN,Pesquisa e leitura de livros,Superior Completo,Solteiro,Outro,Não,Boa no estresse (PC-P),Forte no estresse (PC-P),Moderada no estresse (PC-P),Boa diante do impacto do estresse (PC-P),Boa no estresse (PC-P),Moderada no estresse (PC-P),Excelente no estresse,Boa no estresse (PC-P)
3,13534,NaN,NaN,NaN,Masculino,SP,SP,Administração de Empresas,Analista de Mercado,Superior Completo,Solteiro,Evangélico,Sim,Excelente no estresse,Fraca no estresse (PC-I),Fraca no estresse (PC-I),Forte diante do impacto do estresse (PC-P),Excelente no estresse,Boa no estresse (PC-P),Boa no estresse (PC-P),Boa no estresse (PC-P)
4,13535,Conhecer a comunidade quilombola Pedro Cubas,13 - 17 anos,Mais de um ano,Masculino,SP,SP,NaN,NaN,Superior Incompleto,Solteiro,Evangélico,Sim,Excelente no estresse,Excelente face ao estresse,Excelente no estresse,Excelente face ao impacto do estresse (PC-E),Forte no estresse (PC-P),Boa no estresse (PC-P),Excelente no estresse,Excelente no estresse


In [2419]:
## Verificação de dados duplicados
duplicados = df.duplicated()
print(duplicados)

0       False
1       False
2       False
3       False
4       False
        ...  
2775    False
2776    False
2777    False
2778    False
2779    False
Length: 2780, dtype: bool



**Transformando as colunas**

In [2420]:
df.columns

Index(['Numero do Sujeito', '1a. Doença', 'Idade 1a. Doença',
       'Tempo 1a Doença', 'Sexo', 'UF Nascimento', 'UF', 'Formação',
       'Atividade', 'Escolaridade', 'Estado Civil', 'Religião', 'Outro Idioma',
       'Análise do Contexto', 'Autoconfiança', 'Autocontrole',
       'Conquistar e Manter Pessoas', 'Empatia', 'Leitura Corporal',
       'Otimismo com a Vida', 'Sentido da Vida'],
      dtype='object')

In [2421]:
## Nomes das colunas (todos em minúsculo e sem espaço)
df.columns.str.lower()
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [2422]:
## Renomear colunas relacionadas a doença/trauma
df = df.rename(columns={'1a._doença': 'trauma_doenca', 'idade_1a._doença': 'idade_doenca_trauma', 'tempo_1a_doença': 'tempo_doenca_trauma', 
                        'formação': 'formacao', 'religião': 'religiao', 'análise_do_contexto': 'analise_do_contexto', 'autoconfiança': 'autoconfianca'})

In [2423]:
#Removendo colunas não necessárias
df = df.drop(columns=['numero_do_sujeito', 'atividade'])

**Tratamento de valores nulos (NaN)**

In [2424]:
df = df.dropna(subset=['trauma_doenca', 'idade_doenca_trauma', 'tempo_doenca_trauma'])

**Transformando objetos em categorias**

In [2425]:
#Transformando strings em categorias numéricas
df['sexo'].replace(to_replace = ['Masculino', 'Feminino'], value =[1,2], inplace=True)
df['tempo_doenca_trauma'].replace(to_replace = ['Alguns dias', 'Semanas', 'Meses', 'Um ano', 'Mais de um ano'], value =[1,2,3,4,5], inplace=True)
df['idade_doenca_trauma'].replace(to_replace = ['Até 6 anos', '7 - 12 anos', '13 - 17 anos', '18 - 30 anos', '31 - 49 anos', 'Acima'],
                                  value =[1,2,3,4,5,6], inplace=True)
df['outro_idioma'].replace(to_replace = ['Não', 'Sim'], value =[1,2], inplace=True)

In [2446]:
#Aplicando a moda em valores nulos nas colunas Idade e Tempo de duração da doença ou trauma
df['tempo_doenca_trauma']=df['tempo_doenca_trauma'].fillna(df['tempo_doenca_trauma'].mode()[0])
df['idade_doenca_trauma']=df['idade_doenca_trauma'].fillna(df['idade_doenca_trauma'].mode()[0])

**Padronização do dataframe**


In [2444]:
#Padronizando dataframe em minúsculo e sem acentuação

for column in df.columns:
  df[column] = df[column].astype(str).str.lower()
  
cols = df.select_dtypes(include=[np.object]).columns
df[cols] = df[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))

**Padronizando coluna "Trauma ou Doença"**

In [2428]:
stopwords = ['a', 'minha', 'que', 'me', 'de', 'e', 'na', 'quase', 'dos', 'do', 'meus', 'da', 'aos', 'um', 'numa', 'teve', 'enquanto', 'meu', 
             'sem', 'no', 'para', 'primeira', 'com', 'muito', 'fui', 'durante', 'uma', 'em', 'por', 'o', 'quando', 'eu', 'tive']

In [2429]:
def removestopword (texto):
  semstop = [p for p in texto.split() if p not in stopwords]
  return ' '.join(semstop)

In [2430]:
df['trauma_doenca'] = df['trauma_doenca'].apply(removestopword)

In [2431]:
CATEGORIAS = {'acidente': ['acidente', 'acicente', 'queda', 'capotamento', 'atrolepamento', 'batida', 'atropelamento', 'afogamento', 'carro', 'moto',
              'atropelado'],

              'vicios': ['alcool', 'bebida', 'alcolismo', 'alcoolismo', 'drogas', 'vicio'],

              'violencia domestica ou urbana': ['violencia', 'VIOLÊNCIA', 'violência', 'sequestro', 'arma', 'baleado', 'assassinato', 'agressor',
              'assalto', 'assaltada', 'pedofilia', 'estupro', 'abuso', 'abusado'],

              'doencas leves ou virais': ['enfermidade', 'pneumonia', 'tumor', 'labirintite', 'corte', 'cirurgia', 
              'vesicula', 'queimadura', 'fratura', 'chron', 'hernia', 'hepatite', 'disco', 'bronquite', 'apendicite', 'caxumba', 'tireoide',
              'hipertensao', 'torci', 'artrodese', 'gordura', 'lesao', 'sarampo', 'narcolepsia', 'tenorrafia', 'disfuncao', 'condropatia',
              'asma', 'dor', 'gastrite', 'internacao', 'infarto', 'febre', 'meningite', 'obesidade', 'alzhimer', 'anemia', 'neurolofica',
              'neurologico', 'doenca', 'avc', 'visicula', 'ernia', 'cardiaco', 'edema', 'presao', 'tibia', 'renal', 'operacao', 'braco', 'adoecimento',
              'basite', 'lombares', 'dengue', 'coluna', 'menigite', 'audicao', 'infeccao', 'pericardite', 'catarata', ' inflamacao',
              'operei', 'convulsao', 'abcesso', 'artrite', 'trombose', 'catapora', 'infarte', 'epilepsia', 'torcao', 'deslocar', 'inflamacao', 'renais',
              'quebrei', 'tireoide', 'micose', 'clavicula', 'fraturei', 'diabete', 'prielonefrite', 'rcp', 'cisto', 'amputacao', 'toxoplasmose',
              'comvulcao', 'hemorragia', 'derbuculoce', 'deboculose', 'operasao', 'viral', 'virus', 'internado', 'toc', 'traumatismo', 'alergia',
              'enxaqueca', 'apendice', 'verme', 'visual', 'pylori', 'cardiaca', 'garganta', 'cicatriz', 'sifiles', 'bacteria',
              'gagueira', 'salmonela', 'eclampsia', 'retina', 'gripe', 'taquicardia', 'ictiose', 'entalado', 'endometriose', 'denque', 'quebraduras', 
              'quebrado', 'osso', 'quebrar', 'joelho', 'tornozelo'],

              'doencas graves': ['cancer', 'hiv', 'tuberculose',  'cardiopatia', 'esclerose'],

              'divorcio': ['separacao', 'divorcio', 'separaram', 'conjugal', 'relacionamento', 'traicao'],

              'morte': ['falecimento', 'morte', 'falaceu', 'ente', 'perdas', 'perda', 'obito', 'perca', 'perde', 'falecido'],

              'transtornos ou sindromes': ['ansiedade', 'depressao', 'panico', 'estresse', 'traumatico', 'suicidio', 'disturbio', 'perdi', 'mutilacoes',
              'sociais', 'rejeicao', 'burnout', 'bulimia'],

              'gravidez/maternidade': ['gravidez', 'aborto', 'feto'],

              'outros': ['trabalho', 'escola', 'vestibular', 'preso', 'financeiras', 'desemprego', 'desligado', 'interpessoais', 'namoro', 'nenhuma resposta',
                         'nenhuma', 'nenhum', 'nao houve', '2010', 'nao considero', 'nada', 'nao ha', 'nao recordo', 'ca', 'i', 'nao', '']}

In [2432]:
CATEGORIAS.keys()

dict_keys(['acidente', 'vicios', 'violencia domestica ou urbana', 'doencas leves ou virais', 'doencas graves', 'divorcio', 'morte', 'transtornos ou sindromes', 'gravidez/maternidade', 'outros'])

In [2433]:
def procurar_palavra(lista_palavras, sentenca):
  palavra_presente=False
  for palavra in lista_palavras:
    if palavra in sentenca:
      palavra_presente=True
  return palavra_presente

def mapear_categoria(sentenca):
  for cat in CATEGORIAS.keys():
    if procurar_palavra(CATEGORIAS[cat], sentenca):
      return cat 
  return sentenca

In [2434]:
sentenca = 'violencia dentro casa'
mapear_categoria(sentenca)

'violencia domestica ou urbana'

In [2435]:
df['trauma_doenca'] = df['trauma_doenca'].apply(mapear_categoria)

**Padronizando coluna "Formação"**



In [2436]:
df['formacao'] = df['formacao'].apply(removestopword)

In [2437]:
CAT_FORMACAO = {'militar': ['militar', 'molitar', 'bombeito', 'bombeiro', 'bombeira', 'bombeiros', 'oficial', 'defesa civil', 'bm', 'seguranca', 'cbmdf'],
                'administracao de empresas': ['administradora', 'administracao', 'administrador', 'adm', 'rh', 'empresarial', 'empresaria', 'aministracao',
                'secretaria', 'empresario', 'escritorio', 'negocios', 'secretariado', 'recursos humanos', 'internacionais', 'gestao'],
                'comunicacao': ['propaganga', 'marketing', 'relacoes publicas', 'comunicacao', 'grafica', 'artes', 'designer', 'publicitario', 'publicitaria',
                                'fotografo', 'comunicadora', 'jornalismo', 'telecomunicacoes', 'radiologia', 'jornalista', 'imprensa', 'artes', 'design',
                                'publicidade'],
                'pedagogia': ['professor', 'professora', 'pedagogia', 'letras', 'pedagogo', 'pedagoga'],
                'ciencias contabeis': ['contador', 'contadora', 'contabeis', 'economicas', 'contabil', 'contabilista'],
                'saude': ['medico', 'medica', 'medicina', 'enfermeiro', 'enfermeira', 'enfermagem', 'nutricionista', 'nutricao', 'neurocirurgiao',
                          'farmaceutica', 'clinica', 'biomedicina', 'cirurgia', 'optometria'],
                'arquitetura': ['arquiteto', 'arquiteta', 'arquitetura'],
                'filosofia': ['filosofia', 'filosofo', 'filosofa'],
                'fisica': ['fisico'],
                'historia': ['historia'],
                'geografia': ['geografo', 'geografia'],
                'quimica': ['quimica', 'quimico'],
                'engenharia': ['engenharia', 'engenheiro', 'engenheira', 'eng'],
                'estudante': ['estudante', 'estudanta', 'estudanti', 'estudar', 'escola'],
                'psicologia': ['psicologa', 'psicologo', 'psicologia', 'psicanalista', 'terapia'],
                'economia': ['economia', 'financas', 'economista', 'estatistico'],
                'direito': ['advoga', 'advocacia', 'direito', 'advogada', 'advogado'],
                'tecnologia': ['ti', 'informatica', 'computacao', 'redes', 'sistemas', 'informacao', 'dados', 'sistema', 'desenvolvedor'],
                'outras areas': ['comercial', 'compradora', 'contas', 'odontologia', 'odontologia', 'testador quest', 'metalurgico', 'ambiental', 'pos',
                                 'grau', 'escultor', 'turimologa', 'turismologo', 'oceanografa', 'assistente', 'superior', 'seminarista', 'agro',
                                 'elitricista', 'logistica', 'gastronomia', 'aprendiz', 'mallet', 'operacoes', 'vendas', 'musico', 'coml', 'podologa',
                                 'manicure', 'corretora', 'bancario', 'gerente', 'massoterapeuta', 'biologo', 'biologia', 'turismo', 'arte', 'mecanico',
                                 'mecanica', 'fisioterapeuta', 'juridica', 'educador fisico', 'educacao fisica', 'operadora', 'vendedora', 'eletronica',
                                 'cfo', 'teologia', 'funcionario', 'qualidade', 'teologo', 'condutor.', 'pianista', 'funcionaria', 'nao', 'n', 
                                 'nenhuma', 'nenhum']}

In [2438]:
CAT_FORMACAO.keys()

dict_keys(['militar', 'administracao de empresas', 'comunicacao', 'pedagogia', 'ciencias contabeis', 'saude', 'arquitetura', 'filosofia', 'fisica', 'historia', 'geografia', 'quimica', 'engenharia', 'estudante', 'psicologia', 'economia', 'direito', 'tecnologia', 'outras areas'])

In [2439]:
def procurar_palavra(lista_palavras, sentenca_form):
  palavra_presente=False
  for palavra in lista_palavras:
    if palavra in sentenca_form:
      palavra_presente=True
  return palavra_presente

def mapear_categoria(sentenca_form):
  for cat in CAT_FORMACAO.keys():
    if procurar_palavra(CAT_FORMACAO[cat], sentenca_form):
      return cat 
  return sentenca_form

In [2440]:
sentenca = 'violencia dentro casa'
mapear_categoria(sentenca)

'outras areas'

In [2441]:
df['formacao'] = df['formacao'].apply(mapear_categoria)

In [2445]:
df.head()

,trauma_doenca,idade_doenca_trauma,tempo_doenca_trauma,sexo,uf_nascimento,uf,formacao,escolaridade,estado_civil,religiao,outro_idioma,analise_do_contexto,autoconfianca,autocontrole,conquistar_e_manter_pessoas,empatia,leitura_corporal,otimismo_com_a_vida,sentido_da_vida
0,transtornos ou sindromes,5,4,2,sp,sp,administracao de empresas,pos-graduacao,casado,catolico,2,boa no estresse (pc-p),fraca no estresse (pc-i),fraca no estresse (pc-i),boa diante do impacto do estresse (pc-p),moderada no estresse (pc-i),excelente no estresse,forte no estresse (pc-p),boa no estresse (pc-p)
1,doencas leves ou virais,2,5,1,sp,sp,arquitetura,pos-graduacao,casado,evangelico,2,excelente no estresse,moderada no estresse (pc-i),fraca no estresse (pc-i),boa diante do impacto do estresse (pc-p),boa no estresse (pc-p),boa no estresse (pc-p),excelente no estresse,moderada no estresse (pc-p)
2,violencia domestica ou urbana,2,5,1,sp,sp,outras areas,superior completo,solteiro,outro,1,boa no estresse (pc-p),forte no estresse (pc-p),moderada no estresse (pc-p),boa diante do impacto do estresse (pc-p),boa no estresse (pc-p),moderada no estresse (pc-p),excelente no estresse,boa no estresse (pc-p)
4,outros,3,5,1,sp,sp,outras areas,superior incompleto,solteiro,evangelico,2,excelente no estresse,excelente face ao estresse,excelente no estresse,excelente face ao impacto do estresse (pc-e),forte no estresse (pc-p),boa no estresse (pc-p),excelente no estresse,excelente no estresse
5,doencas graves,3,5,2,sp,sp,comunicacao,superior completo,solteiro,catolico,2,boa no estresse (pc-p),boa no estresse (pc-p),boa no estresse (pc-p),excelente face ao impacto do estresse (pc-e),boa no estresse (pc-p),boa no estresse (pc-p),moderada no estresse (pc-p),boa no estresse (pc-p)
